# Modelos Lineales Generalizados en Python
# Regresión binomial

<img src="https://raw.githubusercontent.com/fhernanb/fhernanb.github.io/master/docs/logo_unal_color.png" alt="drawing" width="200"/>

Aquí se muestran varios ejemplos de como usar Python para ajustar un modelo lineal generalizado. 

Las explicaciones mostradas aquí están basadas en un video de YouTube https://www.youtube.com/watch?v=__oC5IRCFKI

Las librerías necesarias son las siguientes:

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import glm

Otras librerías que se usarán en los ejemplos son:

In [2]:
import pandas as pd

## Ejemplo

En esta actividad vamos a utilizar los datos de los cangrejos presentados la sección 4.1.3 del libro "An Introduction to Categorical Data Analysis" de Agresti (2019). El objetivo es ajustar el siguiente modelo:

\begin{align}
Y_i &\sim Binomial(\mu_i, m=1), \\ 
logit(\mu_i) &= \beta_0 + \beta_1 width_i
\end{align}

El objetivo es modelar ajustar un modelo de regresión logístico para explicar la media $\mu$ de la variable respuesta $Y$ que toma dos valores, $Y=1$ si la cangreja tiene pegados machos a su caparazón y $Y=0$ si la cangreja no tiene machos pegados. La covariable usada en el modelo es $width$ que representa el ancho del caparazón de la cangreja. 

Abajo una figura ilustrativa.

<img src="cangreja_con_un_satelite.png" alt="drawing" width="200"/>

Lo primero que usted debe hacer es leer la base de datos.

In [3]:
file = 'http://www.stat.ufl.edu/~aa/cat/data/Crabs.dat'
datos = pd.read_csv(file, sep='\s+', header=0)
datos.head()

,crab,sat,y,weight,width,color,spine
0,1,8,1,3.05,28.3,2,3
1,2,0,0,1.55,22.5,3,3
2,3,9,1,2.30,26.0,1,1
3,4,0,0,2.10,24.8,3,3
4,5,4,1,2.60,26.0,3,3


Para ver el tamaño de la base de datos

In [4]:
datos.shape

(173, 7)

Para ajustar el modelo:

In [5]:
mod1 = smf.glm(formula='y ~ width', data=datos, 
               family=sm.families.Binomial(link=sm.families.links.logit()))
mod1 = mod1.fit()
mod1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      171
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -97.226
Date:                Thu, 28 Apr 2022   Deviance:                       194.45
Time:                        13:34:16   Pearson chi2:                     165.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -12.3508      2.629     -4.698      0.000     -17.503      -7.199
width          0.4972      0.102      4.887      0.000       0.298       0.697
==============================================================================
"""

Usando los resultados de la tabla anterior podemos escribir el modelo

\begin{align}
Y_i &\sim Binomial(\hat{\mu_i}, m=1), \\ 
logit(\hat{\mu_i}) &= -12.3508 + 0.4972 \, width_i
\end{align}

## Funciones de enlace disponibles

Para conocer otras posibles funciones de enlace se puede utilizar la siguiente instrucción:

In [6]:
sm.families.family.Binomial.links

[statsmodels.genmod.families.links.logit,
 statsmodels.genmod.families.links.probit,
 statsmodels.genmod.families.links.cauchy,
 statsmodels.genmod.families.links.log,
 statsmodels.genmod.families.links.cloglog,
 statsmodels.genmod.families.links.identity]

## Ajustando el modelo usando matrices

En esta parte vamos a volver a ajustar el modelo de regresión logística pero ingresando las matrices $y$ e $X$.

In [7]:
X = datos[["width"]]    # Para crear una matriz con la variable de interés
X = sm.add_constant(X)  # Para adicionar la columna de unos

y = datos[["y"]]

C:\Users\fhern\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [8]:
mod2 = sm.GLM(y, X, family=sm.families.Binomial(link=sm.families.links.logit()))
mod2 = mod2.fit()
mod2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  173
Model:                            GLM   Df Residuals:                      171
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -97.226
Date:                Thu, 28 Apr 2022   Deviance:                       194.45
Time:                        13:34:16   Pearson chi2:                     165.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.3508      2.629     -4.698      0.000     -17.503      -7.199
width          0.4972      0.102      4.887      0.000       0.298       0.697
==============================================================================
"""

Usando los resultados de la tabla anterior podemos escribir el modelo

\begin{align}
Y_i &\sim Binomial(\hat{\mu_i}, m=1), \\ 
logit(\hat{\mu_i}) &= -12.3508 + 0.4972 \, width_i
\end{align}

Vamos ahora a crear la matriz de confusión del modelo para ver el desempeño.

In [9]:
prob_hat = mod2.predict(X)

import numpy as np
y_hat = np.where(prob_hat > 0.5, 1, 0)

from sklearn import metrics
cm = metrics.confusion_matrix(datos["y"], y_hat)
print("La matriz de confusión obtenida es: \n")
print(cm)

La matriz de confusión obtenida es: 

[[27 35]
 [16 95]]


## Haciendo predicciones con el modelo ajustado

Vamos a calcular $P(Y=1 | width)$ para tres cangrejas con anchos de 22, 26 y 28 cm.

Primero vamos a crear la nueva matriz $X$ con los datos.

In [10]:
new_width = {"width" : [22, 26, 28]}
new_X = pd.DataFrame(new_width)
new_X = sm.add_constant(new_X)
print(new_X)

   const  width
0    1.0     22
1    1.0     26
2    1.0     28


C:\Users\fhern\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Ahora vamos a calcular las probabilidades

In [11]:
prob_hat = mod2.predict(new_X)
print(prob_hat)

0    0.195959
1    0.640418
2    0.828017
dtype: float64


Vamos a usar un punto de corte de 0.5 para crear las clasificaciones. Si $P(Y=1) > 0.5$ entonces $Y=1$. 

In [12]:
y_hat = np.where(prob_hat > 0.5, 1, 0)
y_hat

array([0, 1, 1])